In [1]:
!pip install from-root
!pip install emnist
!pip install -U PyYAML

     |████████████████████████████████| 596 kB 5.0 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Shareddrives/Duong-Son/FL-project/fl_learning/
!ls

Mounted at /content/drive
/content/drive/Shareddrives/Duong-Son/FL-project/fl_learning
export_env.sh  main.py	packages	      utils
main.ipynb     model	system_configuration


In [3]:
# from-root
import sys
from from_root import from_root
sys.path.append(str(from_root()))
#
from utils.header import *
from packages.dataset.dataset import JointKDDataset
from packages.engine.derivative_nw import TeacherNetwork, StudentNetwork
from utils.c_reliability import CReliability
from utils.helper_function import *

In [ ]:
start=datetime.now()
# get configure
g_config = GeneralConfigure()
# prepare dataset
dataset = JointKDDataset()
cluster_data = dataset.preProcessData()
list_of_samples_region_train, list_of_samples_region_test = dataset.assignDataForClients(cluster_data)

# get student network
student_model = StudentNetwork()
student  = keras.models.Model(inputs  = student_model.model.input,                      
                              outputs = student_model.model.get_layer('logits').output)
student.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy', 
                metrics=['accuracy'])
# define distilation loss function
distillation_loss_fn = keras.losses.KLDivergence()
# create teacher model
train_able = False
if train_able == True:
  teacher_params = []
  teacher = []
  for region in range(g_config.regions):
    teacher.append(TeacherNetwork(list_of_samples_region_train[region+1],
                                        list_of_samples_region_test[region+1],
                                        list_of_samples_region_train[0],
                                        list_of_samples_region_test[0],
                                        student,
                                        distillation_loss_fn,
                                        g_config.model_path, region, dataset.num_classes))
    teacher_param, _ = teacher[region].regionalAggregation()
    teacher_params.append(teacher_param)
else:
  pass
# C-Reliability 
c_reliability = CReliability(list_of_samples_region_train[0][0], dataset.num_classes)
beta = c_reliability.weightedClass()

dataset_server = processDataServer(list_of_samples_region_train[0][0])

num_of_clients: 0  -  13395
num_of_clients: 1  -  446
num_of_clients: 2  -  3024
num_of_clients: 3  -  3025


In [ ]:
for epoch in range(g_config.distil_epochs):
  print("Start of epoch %d" % (epoch,))
  for step, batch_train in enumerate(dataset_server):
    loss = 0
    x_batch_tf, y_batch_tf = batch_train
    x_batch = x_batch_tf.numpy()
    y_batch = np.argmax(y_batch_tf.numpy(), axis=1)
    # print(len(x_batch))
    # print(len(y_batch))
    for region in range(g_config.regions):
      teacher_model = tf.keras.models.load_model(os.path.join(c_reliability.model_path,f"teacher_region_{region}.h5"),compile=False)
      logit_teacher_model = keras.models.Model(inputs  = teacher_model.input, outputs = teacher_model.get_layer('logits').output)
      logit_teacher_model.compile(optimizer='adam',
                                  loss='sparse_categorical_crossentropy',
                                  metrics=['sparse_categorical_accuracy'])
      logits_predict = logit_teacher_model(x_batch, training=False)
      softmax_predict = softmax(logits_predict)
      rounded_predict = np.argmax(softmax_predict, axis = 1)
      pseudo_dataset = list(zip(x_batch, rounded_predict, y_batch))

      aligned_data_teacher, aligned_label_teacher = dataAlignment(pseudo_dataset, dataset.num_classes)
      teacher_predictions = {}
      for label in range(dataset.num_classes):
        # Calculate individual loss by 
        # Forward pass of teacher
        teacher_predictions[label] = logit_teacher_model(aligned_data_teacher[label].reshape(-1,28,28,1), training=False)
      with tf.GradientTape() as tape:
        # Calculate each label-driven loss
        # Forward pass of student
        for label in range(dataset.num_classes):
          student_predictions = student(aligned_data_teacher[label].reshape(-1,28,28,1), training=True)
          # Compute losses at $label$ round
          # print(aligned_label_teacher[label])
          # print(student_predictions)
          student_loss = g_config.student_loss_fn(aligned_label_teacher[label], student_predictions)
          distillation_loss = beta[region][label] * distillation_loss_fn(tf.nn.softmax(teacher_predictions[label] / 20, axis=1),
                                                      tf.nn.softmax(student_predictions        / 20, axis=1))
          if label == 0: 
              concat_student_predictions = student_predictions
          else:
              concat_student_predictions = tf.concat([concat_student_predictions,student_predictions], 0)
          loss += g_config.alpha * student_loss + (1 - g_config.alpha) * distillation_loss
    # Compute gradients
    trainable_vars = student.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)

    # Update weights
    g_config.optimizer.apply_gradients(zip(gradients, trainable_vars))
    if step % 2 == 0:
      print(f"step {step}: studentloss = {student_loss}, distillation loss = {loss}, accuracy = {g_config.accuracy_metric.result()}")
      student.evaluate(x = list_of_samples_region_test[0][0], y = list_of_samples_region_test[0][1])
stop = datetime.now()
print('Total Time: ', stop - start)

Start of epoch 0


ResourceExhaustedError: ignored

In [4]:
def main():
    start=datetime.now()
    # get configure
    g_config = GeneralConfigure()
    # prepare dataset
    dataset = JointKDDataset()
    cluster_data = dataset.preProcessData()
    list_of_samples_region_train, list_of_samples_region_test = dataset.assignDataForClients(cluster_data)
    
    # get student network
    student_model = StudentNetwork()
    student  = keras.models.Model(inputs  = student_model.model.input,                      
                                 outputs = student_model.model.get_layer('logits').output)
    student.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy', 
                    metrics=['accuracy'])
    # define distilation loss function
    distillation_loss_fn = keras.losses.KLDivergence()
    # create teacher model
    train_able = False
    if train_able == True:
        teacher_params = []
        teacher = []
        for region in range(g_config.regions):
            teacher.append(TeacherNetwork(list_of_samples_region_train[region+1],
                                        list_of_samples_region_test[region+1],
                                        list_of_samples_region_train[0],
                                        list_of_samples_region_test[0],
                                        student,
                                        distillation_loss_fn,
                                        g_config.model_path, region, dataset.num_classes))
            teacher_param, _ = teacher[region].regionalAggregation()
            teacher_params.append(teacher_param)
    else:
        pass
    # C-Reliability 
    c_reliability = CReliability(list_of_samples_region_train[0][0], dataset.num_classes)
    beta = c_reliability.weightedClass()

    dataset_server = processDataServer(list_of_samples_region_train[0][0])
    for epoch in range(g_config.distil_epochs):
        print("Start of epoch %d" % (epoch,))
        for step, batch_train in enumerate(dataset_server):
            loss = 0
            x_batch_tf, y_batch_tf = batch_train
            x_batch = x_batch_tf.numpy()
            y_batch = np.argmax(y_batch_tf.numpy(), axis=1)
            # print(len(x_batch))
            # print(len(y_batch))
            for region in range(g_config.regions):
                teacher_model = tf.keras.models.load_model(os.path.join(c_reliability.model_path,f"teacher_region_{region}.h5"),compile=False)
                logit_teacher_model = keras.models.Model(inputs  = teacher_model.input, outputs = teacher_model.get_layer('logits').output)
                logit_teacher_model.compile(optimizer='adam',
                                            loss='sparse_categorical_crossentropy',
                                            metrics=['sparse_categorical_accuracy'])
                logits_predict = logit_teacher_model(x_batch, training=False)
                softmax_predict = softmax(logits_predict)
                rounded_predict = np.argmax(softmax_predict, axis = 1)
                pseudo_dataset = list(zip(x_batch, rounded_predict, y_batch))

                aligned_data_teacher, aligned_label_teacher = dataAlignment(pseudo_dataset, dataset.num_classes)
                teacher_predictions = {}
                for label in range(dataset.num_classes):
                    # Calculate individual loss by 
                    # Forward pass of teacher
                    teacher_predictions[label] = logit_teacher_model(aligned_data_teacher[label].reshape(-1,28,28,1), training=False)
                with tf.GradientTape() as tape:
                    # Calculate each label-driven loss
                    # Forward pass of student
                    for label in range(dataset.num_classes):
                        student_predictions = student(aligned_data_teacher[label].reshape(-1,28,28,1), training=True)
                        # Compute losses at $label$ round
                        student_loss = g_config.student_loss_fn(aligned_label_teacher[label], student_predictions)
                        distillation_loss = beta[region][label] * distillation_loss_fn(tf.nn.softmax(teacher_predictions[label] / 20, axis=1),
                                                                    tf.nn.softmax(student_predictions        / 20, axis=1))
                        if label == 0: 
                            concat_student_predictions = student_predictions
                        else:
                            concat_student_predictions = tf.concat([concat_student_predictions,student_predictions], 0)
                        loss += g_config.alpha * student_loss + (1 - g_config.alpha) * distillation_loss
            # Compute gradients
            trainable_vars = student.trainable_variables
            gradients = tape.gradient(loss, trainable_vars)

            # Update weights
            g_config.optimizer.apply_gradients(zip(gradients, trainable_vars))       

            if step % 2 == 0:
                print(f"step {step}: studentloss = {student_loss}, distillation loss = {loss}, accuracy = {g_config.accuracy_metric.result()}")
                student.evaluate(x = list_of_samples_region_test[1][0][0], y = np.argmax(list_of_samples_region_test[1][0][1], axis=1))

    stop = datetime.now()
    print('Total Time: ', stop - start)


if __name__ == "__main__":
    os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
    main()

num_of_clients: 0  -  13395
num_of_clients: 1  -  446
num_of_clients: 2  -  3024
num_of_clients: 3  -  3025
Start of epoch 0


KeyError: ignored

In [ ]:
from utils.header import *
from packages.dataset.dataset import JointKDDataset
from packages.engine.derivative_nw import TeacherNetwork, StudentNetwork
from utils.c_reliability import CReliability
from utils.helper_function import *

def main():
    start=datetime.now()
    # get configure
    g_config = GeneralConfigure()
    # prepare dataset
    dataset = JointKDDataset()
    cluster_data = dataset.preProcessData()
    list_of_samples_region_train, list_of_samples_region_test = dataset.assignDataForClients(cluster_data)
    
    # get student network
    student_model = StudentNetwork()
    student  = keras.models.Model(inputs  = student_model.model.input,                      
                                 outputs = student_model.model.get_layer('logits').output)
    student.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy', 
                    metrics=['accuracy'])
    # define distilation loss function
    distillation_loss_fn = keras.losses.KLDivergence()
    # create teacher model
    train_able = False
    if train_able == True:
        teacher_params = []
        teacher = []
        for region in range(g_config.regions):
            teacher.append(TeacherNetwork(list_of_samples_region_train[region+1],
                                        list_of_samples_region_test[region+1],
                                        list_of_samples_region_train[0],
                                        list_of_samples_region_test[0],
                                        student,
                                        distillation_loss_fn,
                                        g_config.model_path, region, dataset.num_classes))
            teacher_param, _ = teacher[region].regionalAggregation()
            teacher_params.append(teacher_param)
    else:
        pass
    # C-Reliability 
    c_reliability = CReliability(list_of_samples_region_train[0][0], dataset.num_classes)
    beta = c_reliability.weightedClass()

    dataset_server = processDataServer(list_of_samples_region_train[0][0])
    for epoch in range(g_config.distil_epochs):
        print("Start of epoch %d" % (epoch,))
        for step, batch_train in enumerate(dataset_server):
            loss = 0
            x_batch_tf, y_batch_tf = batch_train
            x_batch = x_batch_tf.numpy()
            y_batch = np.argmax(y_batch_tf.numpy(), axis=1)
            # print(len(x_batch))
            # print(len(y_batch))
            for region in range(g_config.regions):
                teacher_model = tf.keras.models.load_model(os.path.join(c_reliability.model_path,f"teacher_region_{region}.h5"),compile=False)
                logit_teacher_model = keras.models.Model(inputs  = teacher_model.input, outputs = teacher_model.get_layer('logits').output)
                logit_teacher_model.compile(optimizer='adam',
                                            loss='sparse_categorical_crossentropy',
                                            metrics=['sparse_categorical_accuracy'])
                logits_predict = logit_teacher_model(x_batch, training=False)
                softmax_predict = softmax(logits_predict)
                rounded_predict = np.argmax(softmax_predict, axis = 1)
                pseudo_dataset = list(zip(x_batch, rounded_predict, y_batch))

                aligned_data_teacher, aligned_label_teacher = dataAlignment(pseudo_dataset, dataset.num_classes)
                teacher_predictions = {}
                for label in range(dataset.num_classes):
                    # Calculate individual loss by 
                    # Forward pass of teacher
                    teacher_predictions[label] = logit_teacher_model(aligned_data_teacher[label].reshape(-1,28,28,1), training=False)
                with tf.GradientTape() as tape:
                    # Calculate each label-driven loss
                    # Forward pass of student
                    for label in range(0, dataset.num_classes):
                        if len(aligned_data_teacher[label]) == 0:
                            distillation_loss = 0
                            student_loss = 0
                        else:
                            student_predictions = student(aligned_data_teacher[label].reshape(-1,28,28,1), training=True)
                            student_loss = g_config.student_loss_fn(aligned_label_teacher[label], student_predictions)
                            distillation_loss = beta[region][label] * distillation_loss_fn(tf.nn.softmax(teacher_predictions[label] / 20, axis=1),
                                                                    tf.nn.softmax(student_predictions        / 20, axis=1))
                            
                            if label == 0 or step == 0: 
                                concat_student_predictions = student_predictions
                            else:
                                concat_student_predictions = tf.concat([concat_student_predictions,student_predictions], 0)
                        loss += g_config.alpha * student_loss + (1 - g_config.alpha) * distillation_loss
            # Compute gradients
            trainable_vars = student.trainable_variables
            gradients = tape.gradient(loss, trainable_vars)

            # Update weights
            g_config.optimizer.apply_gradients(zip(gradients, trainable_vars))       

            if step % 2 == 0:
                print(f"step {step}: studentloss = {student_loss}, distillation loss = {loss}, accuracy = {g_config.accuracy_metric.result()}")
                student.evaluate(x = list_of_samples_region_test[1][0][0], y = np.argmax(list_of_samples_region_test[1][0][1], axis=1))

    stop = datetime.now()
    print('Total Time: ', stop - start)


if __name__ == "__main__":
    os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
    main()

num_of_clients: 0  -  13395
num_of_clients: 1  -  446
num_of_clients: 2  -  3024
num_of_clients: 3  -  3025
Start of epoch 0
step 0: studentloss = 3.8191587924957275, distillation loss = 7.71936559677124, accuracy = 0.0
14/14 [==============================] - 1s 33ms/step - loss: 8.2787 - accuracy: 0.0695
step 2: studentloss = 3.7260489463806152, distillation loss = 7.10902214050293, accuracy = 0.0
14/14 [==============================] - 0s 34ms/step - loss: 8.1610 - accuracy: 0.0830
step 4: studentloss = 4.08432674407959, distillation loss = 7.162630558013916, accuracy = 0.0
14/14 [==============================] - 0s 34ms/step - loss: 6.5540 - accuracy: 0.0695
step 6: studentloss = 3.9012537002563477, distillation loss = 6.686339378356934, accuracy = 0.0
14/14 [==============================] - 0s 33ms/step - loss: 5.4848 - accuracy: 0.1996
step 8: studentloss = 3.6225669384002686, distillation loss = 5.72910213470459, accuracy = 0.0
14/14 [==============================] - 0s 34ms